In [1]:
# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 11章　深層学習に挑戦する10本ノック

In [2]:
# 作業フォルダへの移動を行います。
# 人によって作業場所がことなるので、その場合作業場所を変更してください。
import os
os.chdir('/content/drive/MyDrive/11章') #ここを変更。

!pip install tensorflow

from tensorflow.keras import datasets, layers, models
import numpy as np

### 放課後ノック105：物体検出YOLOを使って人の検出を行ってみよう

### YOLOの準備

In [3]:
#yolov3-tf2のダウンロード

''' GitHubから「yolov3-tf2」というフォルダ（ディレクトリ）を丸ごとコピーして、
./yolov3_tf2という名前の新しいフォルダに保存します。→ git cloneはコピーする命令です。'''
!git clone https://github.com/zzh8829/yolov3-tf2.git ./yolov3_tf2

''' 今の作業場所を./yolov3_tf2フォルダに移動します。→ 例えば、机の上にある「yolov3_tf2」箱の中を見るイメージ。'''
%cd ./yolov3_tf2

''' コピーした中のソフトウェアの状態を「c43df87d...」という特定の時点に戻します。
→ Gitはバージョン管理ツールで、開発の履歴を管理しています。→ checkoutはその中から特定の時点を選ぶコマンド。'''
!git checkout c43df87d8582699aea8e9768b4ebe8d7fe1c6b4c

''' ひとつ上のフォルダに戻ります。→ 机の上の箱の外に出るイメージ。'''
%cd ../

fatal: destination path './yolov3_tf2' already exists and is not an empty directory.
/content/drive/MyDrive/11章/yolov3_tf2
M	convert.py
HEAD is now at c43df87 Update dataset.py
/content/drive/MyDrive/11章


In [4]:
#YOLOの学習済みモデルのダウンロード
'''wgetは「ネットからファイルを取ってくる命令」です。YOLOv3-tinyの学習済みの重み（モデル）ファイルのURLです。
実行すると、現在のフォルダに yolov3-tiny.weights という名前で保存されます。'''
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2025-06-04 14:19:56--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 104.21.88.156, 172.67.185.199, 2606:4700:3030::ac43:b9c7, ...
Connecting to pjreddie.com (pjreddie.com)|104.21.88.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘yolov3-tiny.weights.4’

yolov3-tiny.weights     [ <=>                ]   8.88K  --.-KB/s    in 0.005s  

2025-06-04 14:19:56 (1.62 MB/s) - ‘yolov3-tiny.weights.4’ saved [9093]



In [5]:
#ダウンロードしたYOLOの学習済みモデルをKerasから利用出来る形に変換
'''ダウンロードしたYOLOの学習済みモデル（yolov3-tiny.weights）を、TensorFlow/Kerasで使いやすい形（.tfファイル）に変換する処理です。'''
!python ./yolov3_tf2/convert.py --weights ./yolov3-tiny.weights --output  ./yolov3_tf2/checkpoints/yolov3-tiny. --tiny

2025-06-04 14:19:59.538754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749046799.637618    1187 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749046799.670360    1187 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1749046809.207355    1187 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
Model: "yolov3_tiny"
┏━━━━━━━━━━━━━━━

### YOLOによる物体検出の実行

In [6]:
from absl import app, logging, flags
from absl.flags import FLAGS
'''プログラムを動かす前に「このプログラムはyolov3という名前で動きますよ」「コマンドラインで渡されたオプションを読み込みますよ」という準備をしています。'''
app._run_init(['yolov3'], app.parse_flags_with_usage)

['yolov3']

In [8]:
#学習済みの重みをそのまま利用する場合
from  yolov3_tf2.yolov3_tf2.models import  YoloV3Tiny, YoloLoss
import tensorflow as tf
from yolov3_tf2.yolov3_tf2.dataset import transform_images
from yolov3_tf2.yolov3_tf2.utils import draw_outputs
import numpy as np

!python ./yolov3_tf2/convert.py --weights ./yolov3-tiny.weights --output ./yolov3_tf2/checkpoints/yolov3-tiny --tiny

FLAGS.yolo_iou_threshold = 0.5
FLAGS.yolo_score_threshold = 0.5

yolo_class_names = [c.strip() for c in open("./yolov3_tf2/data/coco.names").readlines()]

yolo = YoloV3Tiny(classes=80)
#重みの読み込み
yolo.load_weights("./yolov3_tf2/checkpoints/yolov3-tiny.h5").expect_partial()

2025-06-04 14:57:02.064562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749049022.085053   10615 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749049022.090961   10615 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1749049026.054605   10615 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13714 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
Model: "yolov3_tiny"
┏━━━━━━━━━━━━━━━

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './yolov3_tf2/checkpoints/yolov3-tiny.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
img_filename = "img/img01.jpg"
img_rawP = tf.image.decode_jpeg(open(img_filename, 'rb').read(), channels=3)
data_shape=(256,256,3)
img_yoloP = transform_images(img_rawP, data_shape[0])
img_yoloP = np.expand_dims(img_yoloP, 0)
# 予測開始
boxes, scores, classes, nums = yolo.predict(img_yoloP)

### 結果の出力

In [ ]:
import matplotlib.pyplot as plt

img_yoloP = img_rawP.numpy()
img_yoloP = draw_outputs(img_yoloP, (boxes, scores, classes, nums), yolo_class_names)

plt.figure(figsize=(10,10))
plt.imshow(img_yoloP)
plt.axis('off')
plt.show()

### 放課後ノック106：YOLOの学習を行うための準備をしよう

### 学習データのダウンロード

In [ ]:
#データセットのダウンロード及び解凍を行います。
#ダウンロード済みでない場合以下を実行して下さい。
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!tar -xvf VOCtrainval_06-Nov-2007.tar

### 学習データの確認

In [ ]:
from PIL import Image

#ダウンロードしたデータセットの画像の内１枚を表示
Image.open("./VOCdevkit/VOC2007/JPEGImages/006626.jpg")

In [ ]:
#表示した画像のアノテーションデータの表示
annotation = open("./VOCdevkit/VOC2007/Annotations/006626.xml").read()
print(annotation)

### 放課後ノック107：新たな学習データを使ってYOLOの学習モデルを生成してみよう

### ライブラリのインストール

In [ ]:
!pip install xmltodict

### 学習データの変換

In [ ]:
import xmltodict
import numpy as np
from tensorflow.keras.utils import Sequence
import math
import yolov3_tf2.yolov3_tf2.dataset as dataset

yolo_max_boxes = 100

#アノテーションデータの変換
def parse_annotation(annotation, class_map):
    label = []
    width = int(annotation['size']['width'])
    height = int(annotation['size']['height'])

    if 'object' in annotation:
        if type(annotation['object']) != list:
            tmp = [annotation['object']]
        else:
            tmp = annotation['object']

        for obj in tmp:
            _tmp = []
            _tmp.append(float(obj['bndbox']['xmin']) / width)
            _tmp.append(float(obj['bndbox']['ymin']) / height)
            _tmp.append(float(obj['bndbox']['xmax']) / width)
            _tmp.append(float(obj['bndbox']['ymax']) / height)
            _tmp.append(class_map[obj['name']])
            label.append(_tmp)

    for _ in range(yolo_max_boxes - len(label)):
      label.append([0,0,0,0,0])
    return label

### 学習データ読み込みクラスの定義

In [ ]:
from yolov3_tf2.yolov3_tf2.dataset import transform_images

#学習時に画像データを必要な分だけ読み込むためのクラス
class ImageDataSequence(Sequence):
    def __init__(self, file_name_list, batch_size,  anchors, anchor_masks, class_names, data_shape=(256,256,3)):

        #クラス名とそれに対応する数値、という形の辞書を作る
        self.class_map = {name: idx for idx, name in enumerate(class_names)}
        self.file_name_list = file_name_list

        self.image_file_name_list = ["./VOCdevkit/VOC2007/JPEGImages/"+image_path + ".jpg" for image_path in self.file_name_list]
        self.annotation_file_name_list = ['./VOCdevkit/VOC2007/Annotations/' + image_path+ ".xml" for image_path in self.file_name_list]

        self.length = len(self.file_name_list)
        self.data_shape = data_shape
        self.batch_size = batch_size
        self.anchors = anchors
        self.anchor_masks = anchor_masks

        self.labels_cache = [None for i in range(self.__len__())]

    #１バッチごとに自動的に呼ばれる。画像データとそのラベルを必要な分だけ読み込んで返す
    def __getitem__(self, idx):
        images = []
        labels = []

        #現在のバッチが何回目か、がidx変数に入っているため、それに対応するデータを読み込む
        for index in range(idx*self.batch_size, (idx+1)*self.batch_size):

          #アノテーションデータをラベルとして使える形に変換する
          annotation = xmltodict.parse((open(self.annotation_file_name_list[index]).read()))
          label = parse_annotation(annotation["annotation"], self.class_map)
          labels.append(label)

          #画像データの読み込みと加工
          img_raw = tf.image.decode_jpeg(open(self.image_file_name_list[index], 'rb').read(), channels=3)
          img = transform_images(img_raw, self.data_shape[0])
          images.append(img)

        #ラベルに対しても前処理をするが、時間がかかるため１度読み込んだらキャッシュとして保存する
        if self.labels_cache[idx] is None:
          labels = tf.convert_to_tensor(labels, tf.float32)
          labels = dataset.transform_targets(labels, self.anchors, self.anchor_masks, self.data_shape[0])
          self.labels_cache[idx] = labels
        else:
          labels = self.labels_cache[idx]

        images = np.array(images)
        return images, labels

    def __len__(self):
        return math.floor(len(self.file_name_list) / self.batch_size)



### YOLOモデル(ネットワーク)の読み込み

In [ ]:
import keras
print(keras.__version__)

import tensorflow as tf
print(tf.keras.__version__)

In [ ]:
from  yolov3_tf2.yolov3_tf2.models import  YoloV3Tiny, YoloLoss
from yolov3_tf2.yolov3_tf2.utils import freeze_all
import tensorflow as tf

batch_size=16
data_shape=(416,416,3)
class_names =  ["person", "bird", "cat","cow","dog", "horse","sheep", "aeroplane", "bicycle", "boat", "bus", "car", "motorbike", "train", "bottle", "chair", "diningtable", "pottedplant", "sofa", "tvmonitor"]

anchors = np.array([(10, 14), (23, 27), (37, 58),
                              (81, 82), (135, 169),  (344, 319)],
                             np.float32) / data_shape[0]
anchor_masks = np.array([[3, 4, 5], [0, 1, 2]])

# yolov3_tf2で定義されているtiny YOLOのモデルを読み込む
model_pretrained = YoloV3Tiny(data_shape[0], training=True, classes=80)
model_pretrained.load_weights("./yolov3_tf2/checkpoints/yolov3-tiny.tf").expect_partial()

model = YoloV3Tiny(data_shape[0], training=True, classes=len(class_names))
#ここで、学習済みモデルの出力層以外の重みだけを取り出す
model.get_layer('yolo_darknet').set_weights(model_pretrained.get_layer('yolo_darknet').get_weights())
#出力層以外を学習しないようにする
freeze_all(model.get_layer('yolo_darknet'))

In [ ]:
import tensorflow as tf
print(tf.keras.__version__)
loss = [YoloLoss(anchors[mask], classes=len(class_names)) for mask in anchor_masks]
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss=loss, run_eagerly=False)

#モデルの構造を出力
model.summary()

### 学習データの読み込み

In [ ]:
train_file_name_list = open("./VOCdevkit/VOC2007/ImageSets/Main/train.txt").read().splitlines()
validation_file_name_list = open("./VOCdevkit/VOC2007/ImageSets/Main/val.txt").read().splitlines()

train_dataset = ImageDataSequence(train_file_name_list, batch_size, anchors, anchor_masks, class_names, data_shape=data_shape)
validation_dataset = ImageDataSequence(validation_file_name_list, batch_size, anchors, anchor_masks, class_names, data_shape=data_shape)

### 学習の実施

In [ ]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

In [ ]:
#学習した重みの保存
model.save_weights('./saved_models/model_yolo_weights')

### 放課後ノック108：新たに学習させたモデルを使って人の検出を行ってみよう

### 学習した重みの読み込み

In [ ]:
from absl import app, logging, flags
from absl.flags import FLAGS
app._run_init(['yolov3'], app.parse_flags_with_usage)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from yolov3_tf2.yolov3_tf2.utils import draw_outputs

FLAGS.yolo_iou_threshold = 0.5
FLAGS.yolo_score_threshold = 0.5

yolo_trained = YoloV3Tiny(classes=len(class_names))
#保存した重みの読み込み
yolo_trained.load_weights('./saved_models/model_yolo_weights').expect_partial()

### 物体検出の実行

In [ ]:
img_filename = "img/img01.jpg"

#画像の読み込み
img_rawL = tf.image.decode_jpeg(open(img_filename, 'rb').read(), channels=3)
img_yoloL = transform_images(img_rawL, data_shape[0])
img_yoloL = np.expand_dims(img_yoloL, 0)


#予測開始
boxes, scores, classes, nums = yolo_trained.predict(img_yoloL)

### 結果の表示

In [ ]:
img_yoloL = img_rawL.numpy()

#予測結果を画像に書き込み
img_yoloL = draw_outputs(img_yoloL, (boxes, scores, classes, nums), class_names)

#予測結果を書き込んだ画像の表示
plt.figure(figsize=(10,10))
plt.imshow(img_yoloL)
plt.axis('off')
plt.show()

### 放課後ノック109：YOLOとHOGの人の検出結果を比較して深層学習の精度を体感しよう

### HOGによる人の検出

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# 準備 #
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05, 'hitThreshold':0, 'groupThreshold':5}

# 検出 #
img_hog = cv2.imread("img/img01.jpg")
gray = cv2.cvtColor(img_hog, cv2.COLOR_BGR2GRAY)
human, r = hog.detectMultiScale(gray, **hogParams)
if (len(human)>0):
    for (x, y, w, h) in human:
        cv2.rectangle(img_hog, (x, y), (x + w, y + h), (255,255,255), 3)

# 表示 #
img_hog = cv2.cvtColor(img_hog, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,10))
plt.imshow(img_hog)
plt.axis('off')
plt.show()

### 結果の比較

In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(1,3,1)
plt.title("YOLO (pre-trained)")
plt.imshow(img_yoloP)
plt.axis('off')
plt.subplot(1,3,2)
plt.title("YOLO (trained)")
plt.imshow(img_yoloL)
plt.axis('off')
plt.subplot(1,3,3)
plt.title("HOG")
plt.imshow(img_hog)
plt.axis('off')
plt.show()

### 放課後ノック110：YOLOでの人以外の物体の検出のようすを確認しよう

In [ ]:
img_filename = "img/img02.jpg"

#画像の読み込み
img_rawP = tf.image.decode_jpeg(open(img_filename, 'rb').read(), channels=3)
img_yoloP = transform_images(img_rawP, data_shape[0])
img_yoloP = np.expand_dims(img_yoloP, 0)

#クラス名の読み込み
yolo_class_names = [c.strip() for c in open("./yolov3_tf2/data/coco.names").readlines()]

#予測開始
boxes, scores, classes, nums = yolo.predict(img_yoloP)

#予測結果を画像に書き込み
img_yoloP = img_rawP.numpy()
img_yoloP = draw_outputs(img_yoloP, (boxes, scores, classes, nums), yolo_class_names)

#予測結果を書き込んだ画像の表示
plt.figure(figsize=(10,10))
plt.imshow(img_yoloP)
plt.axis('off')
plt.show()